# Installing libraries

In [190]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pycoingecko

In [1]:
from pycoingecko import CoinGeckoAPI

In [2]:
cg = CoinGeckoAPI()

In [6]:
from datetime import datetime

In [8]:
import pandas as pd

In [16]:
def divide(num,den):
    try:
        result = num/den
    except:
        result = 0
    return result

## Defining functions to get data from the CoinGecko API

In [23]:
def hist_coin(coin):

    hist = cg.get_coin_market_chart_by_id(id=coin, vs_currency='usd', 
                                          days='max', interval='daily')

    i = 0
    hist_price = []
    hist_date = []
    hist_market = []
    growth_p = []
    growth_m = []

    for lin in hist['prices']:
        hist_price.append(lin[1])
        hist_date.append(datetime.fromtimestamp(lin[0]/1000).strftime('%d/%m/%y'))

    for lin in hist['market_caps']:
        hist_market.append(lin[1])

    while i < len(hist['prices']):
        
        res = divide(hist['prices'][i][1],hist['prices'][i-1][1])
        
        if i > 0 and res > 0:
            growth_p.append(res-1)
        else:
            growth_p.append(0)
        i += 1
    
    i=0
    while i < len(hist['market_caps']):
        
        res = divide(hist['market_caps'][i][1],hist['market_caps'][i-1][1])
        
        if i > 0 and res > 0:
            growth_m.append(res-1)
        else:
            growth_m.append(0)
        i += 1
    
    hist_dict = {}
    hist_dict['coin'] = coin
    hist_dict['date'] = hist_date
    hist_dict['price'] = hist_price
    hist_dict['market_cap'] = hist_market
    hist_dict['growth_p'] = growth_p
    hist_dict['growth_m'] = growth_m
    hist_df = pd.DataFrame(hist_dict)

    return hist_df

In [65]:
def hist_data_coin(coin, date_in, date_out):
    
    date_in = datetime.timestamp(datetime.strptime(date_in, '%d/%m/%y'))
    date_out = datetime.timestamp(datetime.strptime(date_out, '%d/%m/%y'))
    
    hist = cg.get_coin_market_chart_range_by_id(id=coin, vs_currency='usd', 
                                                from_timestamp=date_in, 
                                                to_timestamp=date_out)

    i = 0
    hist_price = []
    hist_date = []
    hist_market = []
    growth_p = []
    growth_m = []

    for lin in hist['prices']:
        hist_price.append(lin[1])
        hist_date.append(datetime.fromtimestamp(lin[0]/1000).strftime('%d/%m/%y'))

    for lin in hist['market_caps']:
        hist_market.append(lin[1])

    while i < len(hist['prices']):
        
        res = divide(hist['prices'][i][1],hist['prices'][i-1][1])
        
        if i > 0 and res > 0:
            growth_p.append(res-1)
        else:
            growth_p.append(0)
        i += 1
    
    i=0
    while i < len(hist['market_caps']):
        
        res = divide(hist['market_caps'][i][1],hist['market_caps'][i-1][1])
        
        if i > 0 and res > 0:
            growth_m.append(res-1)
        else:
            growth_m.append(0)
        i += 1
    
    hist_dict = {}
    hist_dict['coin'] = coin
    hist_dict['date'] = hist_date
    hist_dict['price'] = hist_price
    hist_dict['market_cap'] = hist_market
    hist_dict['growth_p'] = growth_p
    hist_dict['growth_m'] = growth_m
    hist_df = pd.DataFrame(hist_dict)

    return hist_df

## Extracting data from de CoinGecko API

In [66]:
# Extracting the historical data of AXS coin
axs = hist_coin('axie-infinity')

In [67]:
axs

,coin,date,price,market_cap,growth_p,growth_m
0,axie-infinity,03/11/20,0.145700,7.754756e+06,0.000000,0.000000
1,axie-infinity,04/11/20,0.145700,7.754756e+06,0.000000,0.000000
2,axie-infinity,05/11/20,0.134893,7.257185e+06,-0.074173,-0.064163
3,axie-infinity,06/11/20,0.142960,7.678065e+06,0.059802,0.057995
4,axie-infinity,07/11/20,0.141995,7.658840e+06,-0.006749,-0.002504
...,...,...,...,...,...,...
341,axie-infinity,10/10/21,116.348095,7.134146e+09,-0.081067,-0.078085
342,axie-infinity,11/10/21,116.051382,7.082348e+09,-0.002550,-0.007260
343,axie-infinity,12/10/21,122.015064,7.463300e+09,0.051388,0.053789
344,axie-infinity,13/10/21,121.909876,7.441777e+09,-0.000862,-0.002884


In [68]:
# Defining the time range analysed
date_in = axs['date'][0]
date_out = axs['date'][len(axs)-1]

In [71]:
# Extracting historical data from ETH coin on the same time range as AXS coin
eth = hist_data_coin('ethereum', date_in, date_out)

In [119]:
# Making a DataFrame with the historical data of the 10 biggest cryptocurrencies in market cap 

df = pd.DataFrame(cg.get_coins_markets(vs_currency='usd', 
                                       order='market_cap_desc', 
                                       per_page=10, page=1, 
                                       price_cange_percentage='200d'))

top_crypto = pd.DataFrame()
crypto = pd.DataFrame()

for ids in df['id']:
    crypto = hist_data_coin(ids, date_in, date_out)
    top_crypto = pd.concat([crypto, top_crypto])

top_crypto = pd.concat([top_crypto, axs])
top_crypto.reset_index(drop=True, inplace=True)

In [162]:
# Making a DataFrame with the historical data of the 10 biggest gaming cryptocurrencies in market cap 

df2 = pd.DataFrame(cg.get_coins_markets(vs_currency='usd', 
                                        order='market_cap_desc', 
                                        category='gaming', 
                                        per_page=10, page=1, 
                                        price_cange_percentage='200d'))

gaming_crypto = pd.DataFrame()
crypto = pd.DataFrame()

for ids in df2['id'].head(10):
    crypto = hist_data_coin(ids, date_in, date_out)
    gaming_crypto = pd.concat([crypto, gaming_crypto])
    
gaming_crypto.reset_index(drop=True, inplace=True)

## Saving the DataFrames in csv files

In [170]:
gaming_crypto.to_csv('./gaming_crypto.csv', sep=';')

In [171]:
top_crypto.to_csv('./top_crypto.csv', sep=';')

In [189]:
axs.to_csv('./axs.csv')

In [322]:
gaming_crypto.sort_values(by='growth_p', ascending=False).head(15)

,coin,date,price,market_cap,growth_p,growth_m
1525,gala,30/12/20,0.001639,1.156355e+07,1.846336,1.660360
1524,gala,29/12/20,0.000576,4.346612e+06,1.377685,1.472809
1782,gala,13/09/21,0.047726,3.624303e+08,1.250431,1.262871
1555,gala,29/01/21,0.003570,2.591946e+07,1.200639,1.080436
989,wax,23/08/21,0.392492,6.022977e+08,1.127584,1.089557
1201,the-sandbox,21/01/21,0.109165,7.300881e+07,1.040219,1.185241
1785,gala,16/09/21,0.102532,7.685048e+08,0.942113,0.941073
2,star-atlas,03/09/21,0.153842,0.000000e+00,0.864014,0.000000
2422,axie-infinity,23/07/21,42.758136,2.364048e+09,0.693661,0.696863
499,illuvium,01/04/21,78.138374,3.216987e+07,0.674089,3.863051


In [319]:
gaming_crypto.groupby('coin').max()

,date,price,market_cap,growth_p,growth_m
coin,,,,,
axie-infinity,31/12/20,135.960860,8.266500e+09,0.693661,0.696863
enjincoin,31/12/20,3.455880,3.237034e+09,0.564554,0.575818
gala,31/12/20,0.117141,8.817959e+08,1.846336,1.660360
illuvium,31/08/21,704.924192,4.476144e+08,0.674089,10.114843
my-neighbor-alice,31/08/21,21.130826,3.729148e+08,0.426239,0.420229
star-atlas,30/09/21,0.233918,5.078566e+08,0.864014,0.228769
the-sandbox,31/12/20,1.007289,8.913814e+08,1.040219,1.185241
verasity,31/08/21,0.062100,4.064256e+08,0.445444,0.419573
wax,31/12/20,0.392492,6.065009e+08,1.127584,1.089557


# Gathering data from Google Trends

In [198]:
from pytrends.request import TrendReq

In [286]:
def search_trend(keyword):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(kw_list=keyword, cat=0, timeframe='today 12-m', geo='', gprop='')
    trend = pytrends.interest_over_time()
    trend.reset_index(inplace=True)
    trend['date'] = pd.to_datetime(trend['date'], format='%d/%m/%Y')
    trend['date'] = trend['date'].dt.strftime('%d/%m/%y')
    return trend

In [278]:
gcoin_trend.reset_index(drop=True, inplace=True)

In [290]:
axs_trend = search_trend(['Axie Infinity'])
ilv_trend = search_trend(['Illuvium'])
gala_trend = search_trend(['Gala'])
star_trend = search_trend(['Star Atlas'])
enjin_trend = search_trend(['EnjinCoin'])

In [296]:
gcoin_trend = pd.merge(axs_trend, ilv_trend)
gcoin_trend = pd.merge(gcoin_trend, gala_trend)
gcoin_trend = pd.merge(gcoin_trend, star_trend)
gcoin_trend = pd.merge(gcoin_trend, enjin_trend)

In [300]:
gcoin_trend.drop(columns='isPartial', inplace=True)

In [301]:
gcoin_trend.to_csv('./gcoin_trend.csv')